<a href="https://colab.research.google.com/github/HelaineMagalhaes/Bot_Mercado_Financeiro_Opcoes_Binarias./blob/main/Bot_Mercado_Financeiro_Opcoes_Binarias.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Importações e Configurações Iniciais
!pip install -U -q google-generativeai
import pandas as pd
import numpy as np


In [3]:
#Get API key Gemini
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('secretName')
genai.configure(api_key=api_key)

In [1]:
# Insira API key ou seu token de acesso do bot do Telegram aqui
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('Telegram')
genai.configure(api_key=api_key)

In [4]:
# Insira API key ou seu token de acesso a PocketOption aqui
import google.generativeai as genai
from google.colab import userdata
api_key = userdata.get('PocketOption')
genai.configure(api_key=api_key)

In [5]:
from pocketoptionapi.stable_api import PocketOption as PocketOptionAPI
import pypocketoption as PocketOptionPy
import telegram
from telegram.ext import Updater, CommandHandler, MessageHandler
import time
import datetime


ModuleNotFoundError: No module named 'pocketoptionapi'

In [ ]:
#Lógica do Operacional
const pocketoption = require('pocketoption');

// Connect to PocketOption API
const io = new pocketoption({
  username: 'your_username',
  password: 'your_password'
});

// Define trading parameters
let asset; // Asset will be selected dynamically
const duration = 1; // Trade duration in minutes
const amount = 0.01; // Trade amount (percentage of bankroll)
const minimumAmount = 1; // Minimum trade amount
const bankroll = 1000; // Initial bankroll

// Define technical indicators
const ema22 = new EMA(22);
const ema9 = new EMA(9);

// Initialize candles array
let candles = [];

// Define trade execution function
function executeTrade(direction) {
  const lastFourCandles = candles.slice(-4); // Get the last 4 candles

  // Check for the trading pattern
  if (lastFourCandles[0].close > lastFourCandles[1].close && // Check if first 2 are red
      lastFourCandles[1].close > lastFourCandles[2].close && // Check if 3rd is green
      lastFourCandles[2].close < lastFourCandles[3].close) { // Check if 4th is red
    if (price < ema22 && price < ema9) { // Check 5th candle price vs EMAs
      direction = 'put'; // Sell (put)
    } else {
      direction = 'call'; // Buy (call)
    }
  }

  // Calculate trade amount based on bankroll and minimum amount
  const tradeAmount = Math.max(amount * bankroll, minimumAmount);

  // Open the trade using PocketOption API
  io.buy(asset, duration, direction, tradeAmount, function(error, result) {
    if (error) {
      console.error(error);
    } else {
      console.log('Trade opened:', result);
    }
  });
}

// Function to get assets with high payout and 1 minute timeframe
function getAssetsWithHighPayoutAnd1MinuteTimeframe() {
  // Implement logic to retrieve assets from PocketOption API
  // Filter assets based on payout and timeframe criteria
  return filteredAssets; // Return list of matching assets
}

// Start monitoring price and executing trades
setInterval(function() {
  // Get a random asset from the list
  asset = getAssetsWithHighPayoutAnd1MinuteTimeframe()[Math.floor(Math.random() * availableAssets.length)];

  io.getInstrumentPrice(asset, function(error, result) {
    if (error) {
      console.error(error);
    } else {
      price = result.price; // Update current price

      // Check if we have 5 candles for signal
      if (candles.length === 5) {
        // Execute trade based on signal and price
        executeTrade(direction);
      }

      // Update candles array
      candles.shift();
      candles.push({
        open: price,
        high: price,
        low: price,
        close: price
      });
    }
  });
}, 60000); // Check price every minute


In [ ]:
#Operar apenas gráficos com payout igual ou superior a 92%
def get_candles(self):
    candles = []
    for instrument in self.instruments:
        # Obter dados de candles para o instrumento atual
        candles_data = self.broker.get_candles(instrument)

        # Calcular payout para cada candle
        for candle in candles_data:
            payout = calculate_payout(candle)

            # Verificar se o payout atende ao requisito
            if payout >= 0.92:
                # Adicionar candle à lista se o payout for igual ou superior a 92%
                candles.append(Candle(instrument, candle))

    return candles


In [6]:
#Gerenciamento de Risco para Não Quebrar a Banca ou Perder Totalmente o Capital
def gerenciamento_risco(capital_inicial, payout, loss_streak=0):
  """
  Calcula os valores de entrada e gerencia o risco com base no método Martingale modificado.

  Args:
    capital_inicial: Capital inicial da conta.
    payout: Payout do ativo selecionado.
    loss_streak: Quantidade de perdas consecutivas (default: 0).

  Returns:
    Uma tupla contendo o valor da entrada, o retorno esperado e a ação a ser tomada.
  """

  # Define o valor da entrada fixa (1% do capital inicial)
  valor_entrada_fixa = 0.01 * capital_inicial

  if loss_streak == 0:
    # Mão fixa (sem perdas anteriores)
    return valor_entrada_fixa, valor_entrada_fixa * payout, "Entrada fixa"

  elif loss_streak == 1:
    # Recuperação da primeira perda
    valor_entrada = 2.09 * valor_entrada_fixa
    return valor_entrada, valor_entrada * payout, "Recuperação 1"

  else:
    # Sequência de perdas, dividindo a entrada em duas mãos
    valor_entrada_1 = 1.54 * valor_entrada_fixa
    valor_entrada_2 = 2.96 * valor_entrada_fixa

    # Calcula o retorno esperado para as duas entradas combinadas
    retorno_esperado = (valor_entrada_1 + valor_entrada_2) * payout - valor_entrada_1

    # Define a ação com base na sequência de perdas
    if loss_streak == 2:
      acao = "Recuperação 2 (Mão 1)"
    elif loss_streak == 3:
      acao = "Recuperação 2 (Mão 2)"
    else:
      # Ajusta o valor da entrada e a ação com base na tabela fornecida
      # (Implementar lógica para entradas posteriores com base na tabela)
      acao = "Recuperação 3+"

    return valor_entrada_1, retorno_esperado, acao

# Exemplo de uso (adaptar para integração com o bot)
capital_inicial = 100  # Obter o valor real da conta da PocketOption ou demo
payout = 0.92  # Obter o payout do ativo selecionado
loss_streak = 2  # Obter a quantidade de perdas consecutivas

valor_entrada, retorno_esperado, acao = gerenciamento_risco(capital_inicial, payout, loss_streak)

print(f"Valor da entrada: ${valor_entrada:.2f}")
print(f"Retorno esperado: ${retorno_esperado:.2f}")
print(f"Ação: {acao}")

Valor da entrada: $1.54
Retorno esperado: $2.60
Ação: Recuperação 2 (Mão 1)
